## MMAT (MULTI-MODAL-AGENT-TRANSLATION)

Partager


Vous avez dit :
Fait moi en markdown un text qui explique les étapes pour avoir accès a AWS. Voici les élements à prendre ne compte
markdown 
### MISE EN ROUTE
#### Installation
https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html
#### Configuration (aws configure)
https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-files.html
ChatGPT a dit :
markdown
Copier
Modifier
# Guide d'accès à AWS

## MISE EN ROUTE

### Installation de l'AWS CLI

Pour interagir avec les services AWS depuis votre terminal, il est recommandé d'installer l'AWS Command Line Interface (CLI). Suivez les étapes ci-dessous pour installer l'AWS CLI sur votre système :

1. **Télécharger l'installateur** :

   - **Linux x86 (64-bit)** :

     ```bash
     curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
     ```

   - **macOS** :

     ```bash
     curl "https://awscli.amazonaws.com/AWSCLIV2.pkg" -o "AWSCLIV2.pkg"
     ```

   - **Windows** :

     Téléchargez le fichier MSI depuis le site officiel d'AWS.

2. **Installer l'AWS CLI** :

   - **Linux** :

     ```bash
     unzip awscliv2.zip
     sudo ./aws/install
     ```

   - **macOS** :

     ```bash
     sudo installer -pkg AWSCLIV2.pkg -target /
     ```

   - **Windows** :

     Exécutez le fichier MSI téléchargé et suivez les instructions à l'écran.

Pour des instructions détaillées et les dernières mises à jour, consultez la documentation officielle : [Installation de l'AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html).

### Configuration de l'AWS CLI

Après l'installation, configurez l'AWS CLI avec vos informations d'identification AWS :

1. **Exécutez la commande de configuration** :

   ```bash
   aws configure
### Sourcs: 
https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html
#### Configuration (aws configure)
https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-files.html

In [15]:
import boto3
import botocore
import chainlit as cl
from langchain_aws import BedrockEmbeddings, ChatBedrock
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from typing import Any, Dict, List
from docx import Document
from docx.oxml import OxmlElement
from docx.oxml.ns import qn


In [16]:
# intiatilization of the bedrock client
client_config = botocore.config.Config(max_pool_connections=100)
bedrock_client = boto3.client(
            "bedrock-runtime", region_name="us-east-1", config=client_config
)

# intialization of the chat model
llm = ChatBedrock(
            client=None,
            model_id="us.meta.llama3-3-70b-instruct-v1:0",
            region_name="us-west-2",
            model_kwargs={"temperature":0},
        )
# intialized the chat prompt
embedding = BedrockEmbeddings(
            client=bedrock_client,
            model_id="amazon.titan-embed-text-v2:0",
            region_name="us-east-1",
        )

2025-03-11 21:35:59 - Found credentials in shared credentials file: ~/.aws/credentials


In [17]:



class TranslatorAgent:
    """
        this class is an example of Translation agent that uses the bedrock model to 
        translate text to a target language.
        It support normal text and JSON object translation.
    """
    def __init__(self, model: str, target_language: str='french'):
        self.model = model
        self.target_language = target_language

    def prompt_template_json_to_json(self):
        return """
            # TRANSLATION RULES
            Your and agent desitinated to translate text to a target language.
            - Do the translation by using the appropriate term in Tender Offer document
            - Ensure the translation is precise and conveys the exact meaning of the original document.
            - Maintain the original formatting and structure of the document to ensure clarity and consistency.
            - Ensure the translation complies with any specific requirements mentioned in the tender documents.

            # INSTRUCTION
            - You will be asked to translate the a \`\`\`JSON\`\`\`\` object to the target language. 
                respect the \`\`\`JSON\`\`\`\`  structure and the special characters.
            - The target language will be mentioned in the prompt.
            - Respect the special characters and formatting of the text.
            - Maintain the original formatting and structure of the document to ensure clarity and consistency.
            - Ensure the translation complies with any specific requirements mentioned in the tender documents
            - Do not add any additional information to the translation. Only translate the text provided. 
            - Do not comment on the text or provide any additional information.
            - If the content is in the target language, do not translate it back to english.

            # TRARGET LANGUAGE
            ----------------------------------------------------------------
            {target_language}
            ----------------------------------------------------------------

            # TEXT TO TRANSLATE
            ----------------------------------------------------------------
            {text_to_translate}
            ----------------------------------------------------------------
            
            # FINAL TRANSLATION
            - Do not add introduction in your response or any additional information, 
            no phrase like 'Voici', 'Voila', 'I think', 'I believe', 'I suggest', etc. 'Voici la traduction' or 'Voilà' are also not allowed.

            # TASK
            Translate the content \`\`\`JSON\`\`\`\`  object in english into  the target language, 
            while preserving the \`\`\`JSON\`\`\`\` object structure and special characters.

        """

    def prompt_template_text_to_text(self):
        return """
# TRANSLATION RULES
You are an agent designated to translate text into a target language.
- Ensure the translation is precise and conveys the exact meaning of the text.
- A good translation is accurate and precise.
- The field of translation concerns the "tender offer" document.

# INSTRUCTION

- The target language will be mentioned in the prompt.
- Respect the special characters and formatting of the text.
- If the text contains incorrectly encoded special characters (e.g., \t, \x06, etc.), ensure the final output is properly formatted and readable.
- Maintain the original formatting and structure of the document to ensure clarity and consistency.
- Ensure the translation complies with any specific requirements mentioned in the tender documents.
- Do not add any additional information to the translation. Only translate the text provided.
- Do not comment on the text or provide any additional information.
- If the content is already in the target language, do not translate it back to English.
- If the text is just a simple string with no actual content to translate (e.g., ": "), return the same string without modification.

# TARGET LANGUAGE
----------------------------------------------------------------
{target_language}
----------------------------------------------------------------

# TEXT TO TRANSLATE
----------------------------------------------------------------
{text_to_translate}
----------------------------------------------------------------

# FINAL TRANSLATION
- Do not add an introduction in your response or any additional information, 
  no phrases like "Voici", "Voilà", "I think", "I believe", "I suggest", etc. 
  Phrases such as "Voici la traduction" or "Voilà" are also not allowed.
- Perform a direct translation of the text into the target language.
- If special characters are misencoded, ensure they are properly interpreted and formatted.
- If the text is only a simple string with no actual content to translate, return it unchanged.

# TASK
Translate the content in English into the target language.
        """
    
    def prompt_template_html_to_html(self):
        return """# TRANSLATION RULES
You are an agent designated to translate text within an HTML document to a target language while maintaining the exact structure and formatting.

Ensure the translation is precise and conveys the exact meaning of the text.
Do not modify, remove, or add any HTML tags, attributes, or special characters.
Do not translate content that appears inside HTML tags (e.g., class names, IDs, attributes like href, src, alt, title).
Preserve the indentation, spacing, and line breaks exactly as in the original document.
If the content is already in the target language, do not translate it.

# INSTRUCTIONS
The target language will be specified in the prompt.
Only translate the text content, not the HTML structure.
Do not add any comments, explanations, or additional information.
Ensure the translation complies with any domain-specific requirements.

# TARGET LANGUAGE
{target_language}

# HTML CONTENT TO TRANSLATE
{text_to_translate}

# FINAL TRANSLATION
Do not add introductions like "Here is the translation," "Voila," etc.
Maintain the exact HTML structure and formatting while translating only the visible text content.

# TASK
Translate the textual content within the HTML while ensuring the integrity of the HTML structure remains unchanged.
"""

    def translate(self, content, parser=StrOutputParser()) -> str:
        
        """
        Translate the given content to the target language.

        Parameters:
        content (str or dict or list): The text or data to be translated. If a dictionary is provided, it will be converted to a list.
        parser (StrOutputParser, optional): An instance of StrOutputParser to process the output. Defaults to StrOutputParser().

        Returns:
        str: The translated text in the target language.

        If the content is empty, an empty string is returned.
        """
        if content != "":
            print("Translating the text to the target language ...")
            if isinstance(content, dict):
                content = [content]
            
            if "<p>" in content:
                print("HTML detected")
                prompt_template = self.prompt_template_html_to_html()
            elif isinstance(content, list):
                prompt_template = self.prompt_template_json_to_json()
            else:
                prompt_template = self.prompt_template_text_to_text()
            
            template = ChatPromptTemplate([
                ("system", prompt_template), 
                ("user", "{text_to_translate}")
            ]) 
            input_dict = {
                "target_language": self.target_language,
                "text_to_translate": content
            }

            chain = template | self.model | parser
            final_answer = chain.invoke(input_dict)
            return final_answer
        return ""

In [18]:
translatior_agent = TranslatorAgent(llm, 'french')


translatior_agent.translate(
"""
    Benjamin F. McAdoo (1920–1981) was an American architect mainly\n
    active in the Seattle area. Born in Pasadena, California, he was\n
    inspired to study architecture by a mechanical-drawing class\n
    and the work of Paul R. Williams.
""")

Translating the text to the target language ...
2025-03-11 21:35:59 - Using Bedrock Invoke API to generate response
2025-03-11 21:36:00 - The message received from Bedrock:     Benjamin F. McAdoo (1920–1981) était un architecte américain principalement

    actif dans la région de Seattle. Né à Pasadena, en Californie, il a été

    inspiré à étudier l'architecture par un cours de dessin mécanique

    et l'œuvre de Paul R. Williams.


"    Benjamin F. McAdoo (1920–1981) était un architecte américain principalement\n\n    actif dans la région de Seattle. Né à Pasadena, en Californie, il a été\n\n    inspiré à étudier l'architecture par un cours de dessin mécanique\n\n    et l'œuvre de Paul R. Williams."

In [19]:
json_content = {
"type_of_classification": "cctp",
"final_answer_classification": "Yes"
}
translatior_agent.translate(json_content)

Translating the text to the target language ...
2025-03-11 21:36:00 - Using Bedrock Invoke API to generate response
2025-03-11 21:36:01 - The message received from Bedrock: [{'type_of_classification': 'cctp', 'final_answer_classification': 'Oui'}]


"[{'type_of_classification': 'cctp', 'final_answer_classification': 'Oui'}]"

In [20]:
# Charger le document Word
doc = Document("file.docx")

# Définir la langue cible (par exemple, français -> anglais)
translatior_agent = TranslatorAgent(llm, 'english')

def get_run_style(run):
    """Retourne un tuple représentant le style d'un run (pour savoir s'il peut être fusionné)"""
    return (
        run.bold,
        run.italic,
        run.underline,
        run.font.name,
        run.font.size,
        run.font.color.rgb
    )

def extract_hyperlinks_with_positions(paragraph):
    """
    Extrait les hyperliens d'un paragraphe et retourne une liste de tuples
    contenant : le texte du lien, l'URL et la position du lien dans le paragraphe.
    """
    links = []
    # position = 0
    
    for index, child in enumerate(paragraph._element):
        if child.tag == qn("w:hyperlink"):
            r_id = child.get(qn("r:id"))
            if r_id:
                part = paragraph.part
                rel = part.rels.get(r_id)
                if rel:
                    url = rel.target_ref
                    link_text = "".join(node.text for node in child.findall(".//" + qn("w:t")) if node.text)
                    links.append((link_text, url, index))
    
    return links

def create_hyperlink(paragraph, text, url, position):
    """Ajoute un hyperlien cliquable à un paragraphe"""
    # Accéder à la partie principale du document
    # document_part = paragraph.part

    # Ajouter une relation de type hyperlien au document
    # r_id = document_part.rels.add_relationship(
    #     "http://schemas.openxmlformats.org/officeDocument/2006/relationships/hyperlink",
    #     url, is_external=True
    # )

    part = paragraph.part
    r_id = part.relate_to(url, "http://schemas.openxmlformats.org/officeDocument/2006/relationships/hyperlink", is_external=True)

    # Créer l'élément XML du lien hypertexte
    hyperlink = OxmlElement("w:hyperlink")
    hyperlink.set(qn("r:id"), r_id)

    # Créer un run avec style (bleu + souligné)
    run = OxmlElement("w:r")
    r_pr = OxmlElement("w:rPr")

    # Ajouter un attribut de couleur (bleu)
    color = OxmlElement("w:color")
    color.set(qn("w:val"), "0000FF")  # Utilisation du bon namespace pour 'val'
    r_pr.append(color)

    # Ajouter l'attribut souligné
    underline = OxmlElement("w:u")
    underline.set(qn("w:val"), "single")  # Souligné simple
    r_pr.append(underline)

    # Ajouter la mise en forme du texte dans le run
    run.append(r_pr)

    # Ajouter le texte cliquable
    text_elem = OxmlElement("w:t")
    text_elem.text = text
    run.append(text_elem)

    # Attacher le run au lien hypertexte
    hyperlink.append(run)
    paragraph._element.insert(position, hyperlink)


def translate_paragraph(para):
    """Fusionne les runs ayant le même style, traduit et réinsère sans perdre la mise en page"""
    if not para.runs:
        return  # Si aucun run, ne rien faire

    new_runs = []
    current_text = ""
    current_style = get_run_style(para.runs[0])

    links = extract_hyperlinks_with_positions(para)
    print(links)

    # Fusionner les runs qui ont le même style
    for run in para.runs:
        if get_run_style(run) == current_style:
            current_text += run.text  # Ajouter le texte du run
        else:
            # Ajouter le run fusionné
            new_runs.append((current_text, current_style))
            current_text = run.text
            current_style = get_run_style(run)
    
    # Ajouter le dernier run fusionné
    if current_text:
        new_runs.append((current_text, current_style))

    # Traduire le texte des runs fusionnés
    translated_runs = [(translatior_agent.translate(text), style) for text, style in new_runs]
    # translated_runs = [(text, style) for text, style in new_runs]
    print(translated_runs)

    # Effacer les runs existants
    para.clear()

    # Réinsérer les nouveaux runs traduits avec le style d'origine
    for text, style in translated_runs:
        new_run = para.add_run(text)
        new_run.bold, new_run.italic, new_run.underline, new_run.font.name, new_run.font.size, new_run.font.color.rgb = style

    for link_text, url, position in links:
        create_hyperlink(para, link_text, url, position)

# Traduire tous les paragraphes
for para in doc.paragraphs:
    translate_paragraph(para)

# Traduire tous les tableaux
for table in doc.tables:
    for row in table.rows:
        for cell in row.cells:
            for para in cell.paragraphs:
                translate_paragraph(para)

# Sauvegarder le document traduit
doc.save("document_traduit2.docx")

[]
Translating the text to the target language ...
2025-03-11 21:36:01 - Using Bedrock Invoke API to generate response
2025-03-11 21:36:02 - The message received from Bedrock: 1SL2000 – SESSION 2024/2025 – INTEGRATED CLASS 4
[('1SL2000 – SESSION 2024/2025 – INTEGRATED CLASS 4', (True, None, None, None, 177800, None))]
[]
Translating the text to the target language ...
2025-03-11 21:36:02 - Using Bedrock Invoke API to generate response
2025-03-11 21:36:02 - The message received from Bedrock: Fairphone Business Model and its Impact on the Mobile Phone Value Chain
[('Fairphone Business Model and its Impact on the Mobile Phone Value Chain', (True, None, None, None, 139700, None))]
[('https://youtu.be/s6TYsjrAbng', 'https://youtu.be/s6TYsjrAbng', 2)]
Translating the text to the target language ...
2025-03-11 21:36:02 - Using Bedrock Invoke API to generate response
2025-03-11 21:36:03 - The message received from Bedrock: Watch the video: , identify the impacts
[('Watch the video: , identify 